# Experiment 0.1.

## Collecting training data from Campus infrastructure

In [1]:
import sys
sys.version

'3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]'

In [8]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
import os

from netunicorn.base import Task, ExperimentStatus, Experiment, Pipeline

from netunicorn.library.tasks.basic import ShellCommand
from netunicorn.library.tasks.upload.webdav import UploadToWebDav
from netunicorn.client.remote import RemoteClient

In [11]:
class BenignTask(Task):
    def __init__(self, url1, url2, *args, **kwargs):
        self.url1 = url1
        self.url2 = url2
        super().__init__(*args, **kwargs)
    
    def run(self):
        import subprocess
        for i in range(500):
            subprocess.check_output(f'curl {self.url1}', shell=True)
            subprocess.check_output(f'curl {self.url2}', shell=True)
        return 0

In [9]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "pinot.cs.ucsb.edu/patator:arm64-v5"

In [12]:
benign_pipeline = Pipeline().then(BenignTask(
    "https://pinot.cs.ucsb.edu/about/",
    "https://pinot.cs.ucsb.edu/netunicorn/health",
))
benign_pipeline.environment_definition.image = "pinot.cs.ucsb.edu/patator:arm64-v5"

In [14]:
client = RemoteClient(
    endpoint=os.environ['NETUNICORN_ENDPOINT'],
    login=os.environ['NETUNICORN_LOGIN'],
    password=os.environ['NETUNICORN_PASSWORD']
)

In [15]:
minion_pool = client.get_nodes()

In [18]:
attackers = {"raspi-e4:5f:01:56:d9:3a", "raspi-e4:5f:01:79:4a:18", "raspi-e4:5f:01:75:6e:53", "raspi-e4:5f:01:56:d6:ce"}
attacker_pool = minion_pool.filter(lambda x: x.name in attackers).take(4)
benign_pool = minion_pool.filter(lambda x: x.name.startswith("raspi") and x.name not in attackers).take(3)

In [19]:
assert len(attacker_pool) == 4
assert len(benign_pool) == 3

In [21]:
experiment = Experiment().map(attacker_pipeline, attacker_pool).map(benign_pipeline, benign_pool)

In [35]:
experiment_label = 'patator-rebuttal-0.2'

In [36]:
client.prepare_experiment(experiment, experiment_label)

'patator-rebuttal-0.2'

In [41]:
client.get_experiment_status(experiment_label)

ExperimentExecutionInformation:
status: ExperimentStatus.RUNNING
experiment: 
 - Deployment: Node=raspi-e4:5f:01:56:d6:ce, executor_id=cccd6fbc-faa8-4494-9d5a-8118436871e3, prepared=True
 - Deployment: Node=raspi-e4:5f:01:75:6e:53, executor_id=e2414edd-a5d5-405a-8bf7-3c9e1baf4b23, prepared=True
 - Deployment: Node=raspi-e4:5f:01:56:d9:3a, executor_id=f221fb19-81f4-4291-be43-e7f8c3b6f4ac, prepared=True
 - Deployment: Node=raspi-e4:5f:01:79:4a:18, executor_id=0e818bc5-d8c5-4e67-992e-58dadd33afb7, prepared=True
 - Deployment: Node=raspi-e4:5f:01:72:a2:eb, executor_id=c4d01d6b-98e3-4c92-8ca1-3755b55d2e6a, prepared=True
 - Deployment: Node=raspi-e4:5f:01:56:d8:cd, executor_id=ce5e546d-5789-4eee-a47b-1dcff18cd396, prepared=True
 - Deployment: Node=raspi-e4:5f:01:56:d9:0a, executor_id=aceeb6ad-0f54-4ef5-9031-29cf339a2068, prepared=True
execution_result:
[DeploymentExecutionResult:
  Node: {self.node}
  Result: <class 'returns.result.Success'>
    08753712-fbf3-4aa9-b33b-234eceeca33b: [<Succes

In [39]:
client.start_execution(experiment_label)

'patator-rebuttal-0.2'

## Collecting testing data from Azure infrastructure

In [16]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

benign_pipeline = Pipeline().then(BenignTask(
    "http://172.174.8.19/about.html",
    "http://172.174.8.19:26511/health",
))
benign_pipeline.environment_definition.image = "redacted"

In [17]:
experiment = Experiment().map(attacker_pool, attacker_pipeline).map(benign_pool, benign_pipeline)

In [18]:
experiment_label = 'patator-external-0.1.1'

In [19]:
client.prepare_experiment(experiment, experiment_label)

'patator-external-0.1.1'

In [20]:
client.start_execution(experiment_label)

'patator-external-0.1.1'

## Collecting testing data from multi-cloud dataset

In [18]:
minion_pool

[netunicorn-search-aws-1, netunicorn-search-aws-2, raspi-e4:5f:01:56:d9:a3, raspi-e4:5f:01:75:6b:2c, raspi-e4:5f:01:56:d6:ce, raspi-dc:a6:32:d7:6e:64, raspi-e4:5f:01:56:d9:3d, raspi-e4:5f:01:75:71:17, raspi-e4:5f:01:56:d9:0a, raspi-e4:5f:01:56:d9:3a, raspi-e4:5f:01:75:6b:35, raspi-e4:5f:01:72:a2:eb, raspi-e4:5f:01:56:d8:cd, raspi-e4:5f:01:56:d5:2b, raspi-e4:5f:01:75:6e:53, raspi-e4:5f:01:56:d9:73, raspi-e4:5f:01:75:54:04, raspi-e4:5f:01:56:d9:8b, raspi-e4:5f:01:72:89:99, raspi-e4:5f:01:79:4a:18, raspi-e4:5f:01:56:d9:a2, raspi-e4:5f:01:56:d8:fc, netunicorn-search-aws-3, netunicorn-search-digitalocean-2, netunicorn-search-digitalocean-1]

In [19]:
attacker_pool = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-1',
    'netunicorn-search-digitalocean-1',
}])
benign_pool = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-2',
    'netunicorn-search-aws-3',
    'netunicorn-search-digitalocean-2',
}])

In [20]:
print(attacker_pool)
print(benign_pool)

[netunicorn-search-aws-1, netunicorn-search-digitalocean-1]
[netunicorn-search-aws-2, netunicorn-search-aws-3, netunicorn-search-digitalocean-2]


In [21]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

benign_pipeline = Pipeline().then(BenignTask(
    "http://172.174.8.19/about.html",
    "http://172.174.8.19:26511/health",
))
benign_pipeline.environment_definition.image = "redacted"

In [22]:
experiment_label = 'patator-multicloud-0.1.2'

In [23]:
experiment = Experiment().map(attacker_pool, attacker_pipeline).map(benign_pool, benign_pipeline)

In [24]:
client.prepare_experiment(experiment, experiment_label)

'patator-multicloud-0.1.2'

In [25]:
client.start_execution(experiment_label)

'patator-multicloud-0.1.2'